In [11]:
import pandas as pd

In [18]:
fact = pd.read_csv('flights_fact.csv', usecols=['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
                                               'scheduledDeparture_key', 'scheduledArrival_key' ,'dateKey',
                                               'on_time', 'delayed', 'early', 'TAIL_NUMBER', 'CANCELLED', 'DIVERTED'],
                  dtype={'scheduledArrival_key' : 'str',
                        'scheduledDeparture_key' : 'str',
                        'dateKey': 'str'})

/var/folders/v7/cd7r3hf1229b9tnytv_qr8hm0000gn/T/ipykernel_81766/2436519184.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  fact = pd.read_csv('flights_fact.csv', usecols=['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',


In [3]:
# keep only the top 10 airports and airlines to make the dataset smaller
top_10_airports = ['ATL', 'LAX', 'ORD', 'DFW', 'DEN', 'JFK', 'SFO', 'SEA', 'MCO', 'LAS']
fact = fact[(fact['ORIGIN_AIRPORT'].isin(top_10_airports)) & (fact['DESTINATION_AIRPORT'].isin(top_10_airports))]

In [4]:
fact

,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,scheduledDeparture_key,scheduledArrival_key,dateKey,early,on_time,delayed
8,AA,N3LAAA,SFO,DFW,0030,0545,20150101,1,0,0
9,DL,N826DN,LAS,ATL,0030,0711,20150101,1,0,0
10,DL,N958DN,DEN,ATL,0030,0523,20150101,1,0,0
21,NK,N632NK,LAS,DFW,0103,0530,20150101,0,1,0
66,UA,N36476,SEA,DEN,0530,0907,20150101,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4869825,UA,N87527,LAX,ORD,2355,0555,20151231,1,0,0
4869830,AS,N570AS,LAX,SEA,2355,0244,20151231,1,0,0
4869834,F9,N701FR,DEN,MCO,2359,0525,20151231,1,0,0
4869842,AA,N3LYAA,LAX,ORD,2359,0603,20151231,1,0,0


In [5]:
fact.sample(5)

,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,scheduledDeparture_key,scheduledArrival_key,dateKey,early,on_time,delayed
1498231,AA,N3JSAA,SFO,ORD,0930,1552,20150427,0,1,0
4129581,WN,N7814B,LAX,LAS,1835,1935,20151104,0,0,1
4434494,UA,N474UA,DFW,DEN,0550,0701,20151128,1,0,0
2724428,F9,N924FR,ATL,LAS,2040,2155,20150725,1,0,0
4157048,AS,N483AS,SEA,LAX,1805,2051,20151106,1,0,0


In [6]:
# fact = fact.sample(frac=0.4, random_state=42)

In [7]:
departure_time = pd.read_csv('departureTime_dim.csv', dtype={'scheduledDeparture_key':'str'})
arrival_time = pd.read_csv('arrivalTime_dim.csv', dtype={'scheduledArrival_key':'str'})
departure_date = pd.read_csv('date_dim.csv', dtype={'dateKey':'str'}, usecols=['MONTH', 'DAY', 'dateKey'])

In [8]:
fact = pd.merge(fact, departure_time, left_on='scheduledDeparture_key', right_on='scheduledDeparture_key')
fact = pd.merge(fact, arrival_time, left_on='scheduledArrival_key', right_on='scheduledArrival_key')
fact = pd.merge(fact, departure_date, left_on='dateKey', right_on='dateKey')

In [9]:
fact.sample(5)

,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,scheduledDeparture_key,scheduledArrival_key,dateKey,early,on_time,delayed,Hour_x,Minute_x,Hour_y,Minute_y,MONTH,DAY
228932,AS,N644AS,SEA,DEN,0830,1209,20150726,1,0,0,8,30,12,9,7,26
290023,NK,N612NK,LAS,DFW,2035,0112,20150829,0,0,1,20,35,1,12,8,29
25113,UA,N14242,ORD,DEN,1735,1912,20150622,0,0,1,17,35,19,12,6,22
5461,F9,N906FR,DEN,ATL,1405,1858,20150509,0,1,0,14,5,18,58,5,9
302593,AS,N527AS,SEA,LAX,2055,2333,20151125,1,0,0,20,55,23,33,11,25


In [167]:
fact.drop(columns=['TAIL_NUMBER', 'scheduledDeparture_key', 'scheduledArrival_key', 'dateKey',
                  'DESTINATION_AIRPORT'], inplace=True)

In [168]:
fact['early'] = fact['early']*0
fact['on_time'] = fact['on_time']*1
fact['delayed'] = fact['delayed']*2

In [169]:
fact['target'] = fact['early'] + fact['on_time'] + fact['delayed']

In [170]:
fact = fact[fact['target'] <= 2]

In [171]:
fact['target'].value_counts() / fact['target'].count()

1    0.458876
0    0.389223
2    0.151901
Name: target, dtype: float64

In [172]:
fact.drop(columns=['early', 'on_time', 'delayed'], inplace=True)
# 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'
fact = pd.get_dummies(fact ,columns=['AIRLINE', 'ORIGIN_AIRPORT'])

In [173]:
fact.sample(5)

,Hour_x,Minute_x,Hour_y,Minute_y,MONTH,DAY,target,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,...,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_DEN,ORIGIN_AIRPORT_DFW,ORIGIN_AIRPORT_JFK,ORIGIN_AIRPORT_LAS,ORIGIN_AIRPORT_LAX,ORIGIN_AIRPORT_MCO,ORIGIN_AIRPORT_ORD,ORIGIN_AIRPORT_SEA,ORIGIN_AIRPORT_SFO
210002,18,6,21,3,7,2,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
241198,11,50,13,36,3,6,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
161235,9,40,17,20,6,16,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
265746,13,45,20,53,11,14,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
63558,6,25,8,24,9,30,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [174]:
Y = fact['target']
fact.drop(columns='target', inplace=True)
X = fact

In [175]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [176]:
# from sklearn import svm
# classifier = svm.SVC(kernel='rbf',
#                     decision_function_shape='ovr')
# classifier.fit(X_train, y_train)

In [177]:
# from sklearn import svm
# from sklearn import metrics
# classifier = svm.SVC(kernel='rbf' , verbose=2)
# classifier.fit(X_train, y_train)

In [178]:
# y_pred = classifier.predict(X_test)
# print(metrics.classification_report(y_test, y_pred, 
#                                     zero_division=1)) # silence warning

In [179]:
# X_train.shape

In [180]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import confusion_matrix, accuracy_score
# from sklearn import metrics
# from sklearn.naive_bayes import GaussianNB

# neigh = GaussianNB()
# neigh.fit(X_train, y_train)
# y_pred = neigh.predict(X_test)

In [181]:
# #Confusion Matrix
# confusion_matrix = confusion_matrix(y_test, y_pred)
# confusion_matrix
# #Calculating Model Accuracy
# accuracy = accuracy_score(y_test, y_pred)*100
# print('Accuracy of the model:' + str(round(accuracy, 2)) + ' %.')

In [182]:
# print(metrics.classification_report(y_test, y_pred))

In [183]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [184]:
# from sklearn.linear_model import LogisticRegression

# # Create the model
# model = LogisticRegression(multi_class='ovr', C=1)

# # Train the model
# model.fit(X_train_scaled, y_train)

# # Make predictions
# y_pred = model.predict(X_test_scaled)

In [185]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.40      0.44     35574
           1       0.49      0.72      0.58     41475
           2       0.00      0.00      0.00     13906

    accuracy                           0.49     90955
   macro avg       0.33      0.38      0.34     90955
weighted avg       0.41      0.49      0.44     90955



/Users/dimitrisbouris/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dimitrisbouris/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dimitrisbouris/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [15]:
fact.groupby(by='AIRLINE')['CANCELLED'].sum()

AIRLINE
AA    0
AS    0
B6    0
DL    0
EV    0
F9    0
HA    0
MQ    0
NK    0
OO    0
UA    0
US    0
VX    0
WN    0
Name: CANCELLED, dtype: int64

In [19]:
fact['DIVERTED'].unique()

array([0])